# Using the FAµST API in Algorithms

After the little tour we've done in the previous notebooks, about the [creation of Faust objects](#creation_links), their [manipulation](#manip_links) and their creation using [factorization](#facto_links) algorithms, we shall see in this fourth notebook how the FAµST API can be deployed seamlessly in algorithms.
Our example, already alluded in the [second notebook](#manip_links), will be the Orthogonal Matching Pursuit algorithm (OMP).

This algorithm intervenes in the dictionary learning problem. Of course, I will not treat the theory behind but I assume the reader is already familiar with.
There is not so much to say so let's go straight to the code example.

### 1. The OMP Algorithm Implementation

In [ ]:
from pyfaust import *
from numpy import zeros, copy, argmax

def tomp(y, D, niter):
    nrows, ncols = D.shape
    x = zeros((ncols,1))
    supp = []
    res = copy(y)
    i = 0
    K = min(nrows, ncols)
    while (len(supp) < K and i < niter):
        j = argmax(abs(D.T*res))
        supp += [j] 
        x[supp,:] = pinv(D[:,supp])*y
        res = y-D[:,supp]*x[supp,:]
        i += 1
    return x

The most important point to notice in this code is that except the import part in the header, all the code seems to be a natural numpy implementation of OMP.  
This is in fact the core philosophy of the FAµST API, as explained in previous notebooks and also in the API documentation, we made sure that a Faust can be seen as a numpy array (or rather as a ``numpy.matrix``) hence this code is in fact totally compatible with the two APIs: the D function argument, which is the dictionary, can be indifferently a ``pyfaust.Faust`` object or a ``numpy.matrix`` object.  
A secondary point is that this implementation is more like a toy concept (as indicated by the "t" in the function name). A more advanced and optimized version is introduced in the [last part of this notebook](#4.-A-Greedy-OMP-Cholesky-Implementation) and in particular allows to define the algorithm stopping criterion according to the error tolerance the user wants.

Next we will test this implementation in both cases. But first, let us define a test case.

### 2. The Test Case Dictionary

For convenience, we shall set up a dictionary which garantees uniqueness of sufficiently sparse representations.
The dictionary is the concatenation of an identity matrix and a Hadamard matrix, and because we work with Faust objects, this concatenation will be a Faust object.

Below is the block matrix of our dictionary:  
$
D =
\left[
\begin{array}{c|c}
I_n & H_n \\
\end{array}
\right]
$

$I_n$ is the identity or Dirac matrix and $H_n$ the orthonormal Hadamard matrix, with n being a power of two.

The condition on which the uniqueness of the sparse representation $x$ of a vector $y$  is insured is defined by the following inequality:  
$ \| x \|_0 < (1 + 1/\mu)/2 $ where $\mu$ denotes the coherence of the dictionary and in case of our specially crafted dictionary $\mu = {1 \over \sqrt n}$.

So let's construct the Faust of D, compute y for a unique sparse x and test our OMP implementation to find out if we effectively retrieve the unique x as we should according to this theorem.

Note that, for a better view and understanding you might consult this article [[1]](#[1]).

In [ ]:
from pyfaust import FaustFactory as FF
from numpy import log2
n = 128
FD = hstack((FF.eye(n),FF.wht(n)))
D = FD.todense()
print(D)

Now that we have our dictionary both defined as a Faust (FD) and as a matrix (D), let's construct our reference sparse vector x, we'll call it $x_0$.

In [ ]:
from numpy import zeros, count_nonzero
from numpy.random import randn, permutation as randperm
from random import randint
from math import floor, sqrt
x0 = zeros((2*n, 1)) # NB: FD.shape[1] == 2*n
nnz = floor(.5*(1+sqrt(n)))
nonzero_inds = randperm(2*n)[:nnz]
# we got nnz indices, now build the vector x0
x0[nonzero_inds,0] = randn()
print("l0 norm of x0: ", count_nonzero(x0))

It remains to compute $y$.

In [ ]:
y = FD*x0

Our test case is complete, we are fully prepared to run the OMP algorithm using a well-defined dictionary as a Faust or as numpy array, this should retrieve our $x_0$ from the vector y. Let's try!

### 3. Running the Algorithm

In [ ]:
x = tomp(y, FD, nnz)
from numpy import allclose
assert(allclose(x,x0))
print("We succeeded to retrieve x0, OMP works!")

We tested OMP on a Faust, go ahead and verify what I was aiming at in the first part of the notebook: is this OMP implementation really working identically on a Faust and a ``numpy.matrix``?

In [ ]:
x = tomp(y, D, nnz)
assert(allclose(x,x0))
print("We succeeded to retrieve x0, OMP works!")

We can conclude that the algorithm is indeed available to both numpy and Faust worlds, and we can imagine surely that other algorithms are reachable through the FAµST API. That's anyway in that purpose that the FAµST library will be extended in the future (starting from the latest version which is 2.5.1).

### 4. An OMP-Cholesky Implementation

Speaking of the OMP algorithm and the possibility to implement other optimization algorithms with FAµST, it would be a pity not to mention that the library is delivered with another implementation of OMP.  
This implementation is actually an optimized version which takes advantage of the Cholesky factorization to simplify the least-square problem to solve at each iteration.
This algorithm is implemented into the ``tools`` module of pyfaust.

In [ ]:
from pyfaust.tools import omp
help(omp)

This implementation is integrated into pyfaust as a tool for the Brain Source Localization demo which is documented [here](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/classpyfaust_1_1demo_1_1bsl.html).  
To show off a little, let's run this demo:

In [ ]:
%%capture 
from pyfaust.demo import bsl
bsl.run() # it will take some time (sorry), many Faust-s are compared to the original MEG matrix

In [ ]:
%%capture --no-display
%matplotlib inline
from pyfaust.demo import bsl
bsl.fig_time_cmp()

What we see in this figure is that it takes a few dozens of milliseconds (the median time) to compute the BSL experiment on the the matrix M. This is well above the time it takes with Faust approximates $\hat M_6$ to $\hat M_{23}$ in which the numbers 6 and 23 denote the Faust [RCG](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/classpyfaust_1_1Faust.html#a6a51a05c20041504a0b8f2a73dd8d05a). The greater the RCG the better the computation time is, as we already saw in the [notebook](#manip_links) about Faust manipulations.

As a complementary test, let's verify that the two runs of ``omp()`` on FD and D we constructed before for the toy omp give the same results even if the vector to retrieve is not sparse. Here for instance, $ \| x_1 \|_0 = 98 $

In [ ]:
nnz = 98
x1 = zeros((2*n, 1))
nnz_inds = randperm(2*n)[:nnz]
x1[nnz_inds, 0] = randn()
y = FD*x1
x2 = omp(y, D, maxiter=nnz-1)
x3 = omp(y, FD, maxiter=nnz-1)
# verify if the solutions differ
print("Are x2 and x3 solutions almost equal?", norm(x2-x3)/norm(x3) < 10**-12)
print("Is x1 retrieved into x2?", allclose(x1,x2))
print("Is x1 retrieved into x3?", allclose(x1,x3))
print("x3 mse: ",norm(y-D*x2)**2/y.shape[0])
print("x2 mse: ",norm(y-FD*x3)**2/y.shape[0])
print("x2 and x3 nnz:", count_nonzero(x2), count_nonzero(x3))

As expected, we didn't retrieve our starting x1 (the reason is the condition already discussed in 1.). However we did even better by finding sparser representations. Anyway let's mention that here again like it was with the toy OMP it works the same with the Faust API or with numpy.

Finally, let's check the compute time for applying our dictionary to a vector both for the numpy and Faust versions.

In [ ]:
%timeit D*x2
%timeit FD*x3

***The fourth notebook is ending here***, I hope you'll be interested in trying yourself to write another algorithm with the FAµST API and maybe discovering any current limitation. Don't hesitate to contact us in that case, we'll appreciate any feedback!

### Links

<a name="creation_links">Faust creation (1st) notebook: </a> [html](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/Faust_creation.html), [ipynb](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/Faust_creation.ipynb)  
<a name="manip_links">Faust manipulation (2nd) notebook:</a> [html](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/Faust_manipulation.html), [ipynb](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/Faust_manipulation.ipynb)  
<a name="facto_links">Factorization (3rd) notebook: </a>[html](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/Faust_factorization.html), [ipynb](https://faustgrp.gitlabpages.inria.fr/faust/last-doc/html/Faust_factorization.ipynb)  
<a name="[1]">[1]</a> [Tropp, J. A. (2004). Greed is Good: Algorithmic Results for Sparse Approximation. IEEE Transactions on Information Theory, 50(10), 2231–2242](http://doi.org/10.1109/TIT.2004.834793).